In [1]:
import feedparser
import pandas as pd
from textblob import TextBlob
import psycopg2
from datetime import datetime

# Function to fetch data from the RSS feed
def fetch_feed_data(feed_url):
    feed = feedparser.parse(feed_url)
    entries_list = []
    for entry in feed.entries[:10]:
        blob = TextBlob(entry.title)
        sentiment = blob.sentiment
        polarity = round(sentiment.polarity, 2)
        subjectivity = round(sentiment.subjectivity, 2)
        entries_list.append({
            "Title": entry.title,
            "Polarity": polarity,
            "Subjectivity": subjectivity,
            "Published": entry.published,
            "Link": entry.link
        })
    return entries_list

# Function to create a connection to the database
def create_db_connection(host, port, dbname, user, password):
    try:
        conn = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        print("Database connection successful!")
        return conn
    except Exception as e:
        print(f"Error connecting to the database: {e}")
        return None

# Function to insert data into the database
def insert_data_into_db(conn, entries_list):
    try:
        cur = conn.cursor()
        insert_query = """
        INSERT INTO news_articles (title, polarity, subjectivity, published, link) 
        VALUES (%s, %s, %s, %s, %s);
        """
        for entry in entries_list:
            # Convert the published date to TIMESTAMP format
            published_date = datetime.strptime(entry['Published'], "%a, %d %b %Y %H:%M:%S %Z")
            cur.execute(insert_query, (
                entry['Title'], 
                entry['Polarity'], 
                entry['Subjectivity'], 
                published_date, 
                entry['Link']
            ))
        conn.commit()
        print("Data inserted into the database successfully!")
    except Exception as e:
        print(f"Error inserting data: {e}")
    finally:
        cur.close()
        conn.close()

# RSS feed URL
feed_url = "https://news.google.com/rss/search?q=italy=en"

# Database connection details
db_config = {
    'host': 'database-1.cv8ui6imk0j3.eu-north-1.rds.amazonaws.com',
    'port': '5432',
    'dbname': 'first_db',
    'user': 'fb',
    'password': 'Fulbru84_'
}

if __name__ == "__main__":
    # Fetch data from the RSS feed
    entries = fetch_feed_data(feed_url)

    # Create a connection to the database
    db_conn = create_db_connection(**db_config)

    if db_conn:
        # Insert data into the database
        insert_data_into_db(db_conn, entries)


Database connection successful!
Data inserted into the database successfully!
